In [ ]:
## Project to read in all tours from explore.co.uk
# Use this output to be able to recommend tours given a request from a user

In [ ]:
# start the llm
#> ollama run llama3

In [ ]:
# import libraries

In [ ]:
import json
import requests
import html5lib
import bs4
from bs4 import BeautifulSoup 

In [ ]:
## define functions to be used

In [ ]:
# function for calculating similarities

In [ ]:
def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [ ]:
# function to return similarities of documents

In [ ]:
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = jaccard_similarity(user_input, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]

In [ ]:
# read in a list of urls to scrape the tour details from

In [ ]:
tour_urls = []
corpus_of_documents = {}

In [ ]:
# iterate through the list and get the itinery for each tour

In [ ]:
for url in tour_urls:
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')

    tour = soup.title
    table = soup.find('div', attrs = {'id':'itinerary'})

    itinerary = []  # a list to store quotes
    for row in table.findAll('div', attrs = {'class':'pr-i-desc'})): 
        itinerary.append(row.p)

    corpus_of_documents['author'] = itinerary


In [ ]:
# define corpus

In [ ]:
# user input = ""

In [ ]:
# find the relevant document

In [ ]:
relevant_document = return_response(user_input, corpus_of_documents)

In [ ]:
# create prompt

In [ ]:
prompt = """
You are a bot that makes recommendations for holidays. You answer with facts, highlighting pros and cons.
This is the recommended holiday: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""

In [ ]:
# setup llama3
url = 'http://localhost:11434/api/generate'
data = {
    "model": "llama3",
    "prompt": prompt.format(user_input=user_input, relevant_document=relevant_document)
}
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)

In [ ]:
# generate and return the response

In [ ]:
full_response = []
try:
    count = 0
    for line in response.iter_lines():
        # filter out keep-alive new lines
        # count += 1
        # if count % 5== 0:
        #     print(decoded_line['response']) # print every fifth token
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            
            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

In [ ]:
filename = 'inspirational_quotes.csv'
with open(filename, 'w', newline='') as f:
    w = csv.DictWriter(f,['theme','url','img','lines','author'])
    w.writeheader()
    for quote in quotes:
        w.writerow(quote)

In [ ]:
# close llama3
#> /bye